In [ ]:
import os
import requests
import pandas as pd
import time
from tqdm import tqdm
from datetime import datetime, timedelta
import ssl
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from dotenv import load_dotenv
load_dotenv()

# 연결 테스트

In [ ]:
import requests
import urllib.parse
import ssl
import warnings
import os
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from dotenv import load_dotenv
from pprint import pprint
import json

# 라이브러리 및 API 키 설정
load_dotenv()
warnings.filterwarnings('ignore', category=InsecureRequestWarning)
ssl._create_default_https_context = ssl._create_unverified_context

API_KEY = os.getenv('DO_API_KEY')
BASE_URL = 'http://apis.data.go.kr/B552845/katSale/trades'

# API 연결 테스트
def test_connection() -> bool:
    """API 연결 테스트"""
    params = {
        'serviceKey': API_KEY,
        'pageNo': 1,
        'numOfRows': 1
    }
    
    try:
        response = requests.get(BASE_URL, params=params, verify=False, timeout=15)
        return response.status_code == 200
    except requests.exceptions.RequestException:
        return False

# 데이터 조회
def get_market_data(date: str, market_code: str, large_cat: str, mid_cat: str) -> dict:
    """시장 데이터 조회"""
    params = {
        'serviceKey': API_KEY,
        'pageNo': 1,
        'numOfRows': 10,
        'cond[trd_clcln_ymd::EQ]': date,
        'cond[whsl_mrkt_cd::EQ]': market_code,
        'cond[gds_lclsf_cd::EQ]': large_cat,
        'cond[gds_mclsf_cd::EQ]': mid_cat
    }
    
    try:
        response = requests.get(BASE_URL, params=params, verify=False, timeout=15)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API 요청 실패: {e}")
        return None

# 실행
if __name__ == '__main__':
    if not API_KEY:
        print("오류: .env 파일에 API 키(DO_API_KEY)가 설정되지 않았습니다.")
        exit()
        
    if test_connection():
        print("API 연결 성공")
        
        # 데이터 조회
        result = get_market_data(date='2024-01-15', market_code='110001', large_cat='06', mid_cat='01')
        
        if result:
            print("데이터 조회 성공")
            pprint(result)
        else:
            print("데이터 조회 실패")
            # 다른 날짜로 재시도
            print("\n다른 날짜로 재시도...")
            for test_date in ['2023-12-15', '2023-11-15']:
                result = get_market_data(date=test_date, market_code='110001', large_cat='06', mid_cat='01')
                if result:
                    print(f"재시도 성공 ({test_date})")
                    pprint(result)
                    break
    else:
        print("API 연결 실패. API 키 또는 네트워크 설정 확인")

# 추출코드

In [ ]:
import pandas as pd
import requests
import os
import time
import warnings
import ssl
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from dotenv import load_dotenv
from datetime import datetime, timedelta
from tqdm import tqdm

# SSL 및 경고 설정
warnings.filterwarnings('ignore', category=InsecureRequestWarning)
ssl._create_default_https_context = ssl._create_unverified_context

# 디렉토리 준비
os.makedirs("data/fail_log", exist_ok=True)

# API 설정
load_dotenv()
API_KEY = os.getenv('DO_API_KEY')
BASE_URL = 'http://apis.data.go.kr/B552845/katSale/trades'

if not API_KEY:
    raise ValueError("환경변수 'DO_API_KEY'가 설정되지 않았습니다.")

# 도매시장 코드 및 품목 코드 로드
try:
    df_market = pd.read_csv('도매시장_코드.csv', encoding='cp949')
    df_date = pd.read_csv("holiday_score_table.csv", encoding="cp949")
    holidays = df_date[df_date['holiday_flag'] == 1]['date'].values
except FileNotFoundError as e:
    print(f"오류: 필수 파일을 찾을 수 없습니다. {e.filename}")
    exit()

# 품목 코드 설정
ITEM_CODES = {
#    "양파": "1201",  # 진성
#     "배추": "1001",  # 용곤
#     "상추": "1005",  # 동현
     "사과": "0601",   # 재성
#     "무" : '1101',
#     "감자" : '0501',
#    "대파" : "1202",     # 진성
#     "건고추" : "1207",   # 용곤
#     "마늘" : "1209",  # 동현
     "딸기" : '0804',   # 재성
#     "방울토마토" : "0806",  # 진성
#     "오이" : "0901",
#     "양배추" : "1004",
#     "고구마" : '0502',
#     '배' : '0602'
}

# 날짜 범위 설정
start_date_str = '2018-01-01'
end_date_str = '2025-05-31'
start_dt = datetime.strptime(start_date_str, '%Y-%m-%d')
end_dt = datetime.strptime(end_date_str, '%Y-%m-%d')
total_days = (end_dt - start_dt).days + 1

# 기타 설정
max_retries = 3
FAIL_LOG = []
df_fail_all = pd.DataFrame()

# 품목별 데이터 수집
for item_name, code in tqdm(ITEM_CODES.items(), desc="전체 품목 진행"):
    LARGE = code[:2]
    MID = code[2:]
    data_list = []
    
    current_dt = start_dt
    
    while current_dt <= end_dt:
        date_str = current_dt.strftime('%Y-%m-%d')
        
        # 휴일이면 건너뛰기
        if date_str in holidays:
            current_dt += timedelta(days=1)
            continue
            
        for mcode, market_name in df_market.values:
            # 특정 시장 코드 제외
            if str(mcode) == '210005':
                continue
                
            retry_count = 0
            while retry_count < max_retries:
                try:
                    page_no = 1
                    while True:
                        params = {
                            'serviceKey': API_KEY,
                            'pageNo': page_no,
                            'numOfRows': 300,
                            'cond[trd_clcln_ymd::EQ]': date_str,
                            'cond[whsl_mrkt_cd::EQ]': mcode,
                            'cond[gds_lclsf_cd::EQ]': LARGE,
                            'cond[gds_mclsf_cd::EQ]': MID
                        }
                        
                        response = requests.get(BASE_URL, params=params, verify=False, timeout=10)
                        response.raise_for_status()
                        
                        json_data = response.json()
                        body = json_data.get('response', {}).get('body', {})
                        items = body.get('items', {}).get('item', [])
                        
                        if isinstance(items, dict):
                            items = [items]
                        
                        if not items:
                            break
                        
                        data_list.extend(items)
                        
                        if len(items) < 300:
                            break
                        
                        page_no += 1
                    break
                    
                except requests.exceptions.RequestException as e:
                    retry_count += 1
                    if retry_count == max_retries:
                        FAIL_LOG.append({
                            "item": item_name,
                            "market": market_name,
                            "mcode": mcode,
                            "date": date_str,
                            "reason": str(e)
                        })
                    time.sleep(1) # 재시도 전 대기

            time.sleep(0.1) # API 부하 방지
            
        current_dt += timedelta(days=1)
    
    # 품목별 결과 저장
    os.makedirs(f"data/{item_name}", exist_ok=True)
    
    if data_list:
        df = pd.DataFrame(data_list)
        filename = f"data/{item_name}/유통공사_도매시장_{item_name}_{start_date_str.replace('-', '')}-{end_date_str.replace('-', '')}.csv"
        df.to_csv(filename, encoding='cp949', index=False)
        print(f"\n- {item_name}: {len(df)}건 데이터 저장 완료")
    else:
        print(f"\n- {item_name}: 수집된 데이터 없음")

    # 품목별 실패 로그 저장
    if FAIL_LOG:
        df_fail = pd.DataFrame(FAIL_LOG)
        df_fail_all = pd.concat([df_fail_all, df_fail], ignore_index=True)
        fail_filename = f"data/{item_name}/유통공사_fail_log_{item_name}_{start_date_str.replace('-', '')}-{end_date_str.replace('-', '')}.csv"
        df_fail.to_csv(fail_filename, index=False, encoding='cp949')
        print(f"  {len(FAIL_LOG)}건의 실패 로그 저장")

print("\n모든 품목 데이터 수집 완료")

if not df_fail_all.empty:
    df_fail_all.to_csv("data/fail_log/총_실패_로그.csv", index=False, encoding='cp949')
    print(f"총 {len(df_fail_all)}건의 실패 로그가 'data/fail_log'에 저장되었습니다.")